In [1]:
import geopandas as gp
from shapely.geometry import Point
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from joblib import Parallel,delayed
import os

In [3]:
def generate_noise(p):
    noise_point = None
    b_bounds = p.geometry.bounds
    while noise_point == None:
        random_point = Point(random.uniform(b_bounds[0], b_bounds[2]), random.uniform(b_bounds[1], b_bounds[3]))
        if ((p.geometry.contains(random_point))&(land_polygon.contains(random_point))).values:
            noise_point = random_point
    return noise_point

In [3]:
def stream(df):
    #print(df)
    point_shp = df.in_path
    geo_df = gp.read_file(point_shp,encoding = 'gb18030')#.head(10)
    geo_df1 = geo_df.copy()
    geo_df1['geometry'] = geo_df.geometry.buffer(np.array([d]*len(geo_df)))
    geo_df1 = geo_df1.to_crs(epsg=4326)
    #geo_df.buffer_p = geo_df.buffer_p.to_crs(epsg=4326)
    geo_df1['noise_point'] = geo_df1.apply(generate_noise,axis=1)
    
    out_df = geo_df1[['date', 'lon', 'lat','type','noise_point']]
    out_df['lon'] = out_df.noise_point.apply(lambda x:x.x)
    out_df['lat'] = out_df.noise_point.apply(lambda x:x.y)
    #out_df=out_df.drop(columns=['noise_point'])
    
    out_df.to_excel(df.out_path,index=False)
    return out_df

In [4]:
def tmp_func(df):
    index_value = df.apply(stream,axis=1)
    return index_value

def apply_parallel(df_grouped,func):
    results = Parallel(n_jobs=10)(delayed(func)(group) for name,group in df_grouped)
    return pd.concat(results)

In [5]:
land_path = r'shpfile/mask_land.shp'
land_polygon = gp.read_file(land_path).geometry

In [ ]:
## bird gps points was added different intensity noise in spatial, after rebuild the dataset(point-environment_variable) and train,generate the figure2b

In [6]:
for d in [5000,10000,20000,50000,100000]:
    in_list = []
    out_list = []
    d_list = []
    for i in range(0,50):
        temp_in = r'shpfile/west.shp'
        temp_out = r'noise-added-point/noise/noise_%skm/%s/gpsdata_withnoise/translated/west.xlsx'%(int(d/1000),i)
        #break
        d_list.append(d)
        in_list.append(temp_in)
        out_list.append(temp_out)
        #stream(inpath=temp_in,outpath=temp_out)
    path_df = pd.DataFrame([in_list,out_list,d_list]).T
    path_df.columns = ['in_path','out_path','distance']
    df_grouped = path_df.groupby(path_df.index)
    #print(1)
    out = apply_parallel(df_grouped,tmp_func)
    #break

In [7]:
for d in [5000,10000,20000,50000,100000]:
    in_list = []
    out_list = []
    d_list = []
    for i in range(0,50):
        temp_in = r'shpfile/east.shp'
        temp_out = r'noise-added-point/noise/noise_%skm/%s/gpsdata_withnoise/translated/east.xlsx'%(int(d/1000),i)
        #break
        d_list.append(d)
        in_list.append(temp_in)
        out_list.append(temp_out)
        #stream(inpath=temp_in,outpath=temp_out)
    path_df = pd.DataFrame([in_list,out_list,d_list]).T
    path_df.columns = ['in_path','out_path','distance']
    df_grouped = path_df.groupby(path_df.index)
    #print(1)
    out = apply_parallel(df_grouped,tmp_func)
    #break